In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from environment import *
from setting import *

In [ ]:
w = pd.read_table(PATH_DICT["w_file_path"], index_col=0)

h = pd.read_table(PATH_DICT["h_file_path"], index_col=0)

In [ ]:
from numpy import absolute, isnan, where
from scipy.spatial.distance import correlation


def compute_correlation_distance(x, y):

    correlation_distance = correlation(x, y)

    if isnan(correlation_distance):

        return 2

    else:

        return where(absolute(correlation_distance) < 1e-8, 0, correlation_distance)

In [ ]:
for w_or_h, df in (("w", w.T), ("h", h)):

    k_hcc = ccal.hierarchical_consensus_cluster_with_multiple_k(
        df,
        HCC_KS,
        n_job=MAX_N_JOB,
        distance_function=compute_correlation_distance,
        plot_df=False,
        directory_path=PATH_DICT["{}_hcc_directory_path".format(w_or_h)],
    )

    ks = tuple("K{}".format(k) for k in HCC_KS)

    hcc__k_x_column = pd.read_table(
        PATH_DICT["{}_hcc__k_x_column_file_path".format(w_or_h)], index_col=0
    )

    for k, element_cluster in hcc__k_x_column.iterrows():

        if w_or_h == "w":

            xaxis_title = FEATURE_NAME

        elif w_or_h == "h":

            xaxis_title = SAMPLE_NAME

        if df.shape[1] < 3200:

            ccal.plot_heat_map(
                df,
                normalization_axis=0,
                normalization_method="-0-",
                column_annotation=element_cluster,
                title=k,
                xaxis_title=xaxis_title,
                yaxis_title="P",
                html_file_path="{}/{}.html".format(
                    PATH_DICT["{}_hcc_directory_path".format(w_or_h)], k
                ),
            )